In [119]:
# In this code we are implementing ANN(Artificial neural network) / MLP(Multi layer perceptron)
import pandas as pd
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim

In [120]:
from google.colab import files
files.upload()

{}

In [121]:
df = pd.read_csv("fmnist_small.csv")

In [122]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [123]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [124]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state= True)

In [125]:
X_train = X_train/255
X_test = X_test/255

In [126]:
class customdataset(Dataset):
  def __init__(self,features,label):
    self.features = torch.tensor(features.values, dtype = torch.float32)
    self.label = torch.tensor(label.values, dtype = torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self, index):
    return self.features[index] , self.label[index]

In [127]:
train_dataset = customdataset(X_train,y_train)

In [128]:
test_dataset = customdataset(X_test,y_test)

In [129]:
train_loader = DataLoader(train_dataset , batch_size = 32 , shuffle = True)

In [130]:
test_loader = DataLoader(test_dataset , batch_size = 32 , shuffle = False)

In [131]:
class ANN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, X):
        return self.model(X)


In [132]:
 epochs= 100
 learning_rate = 0.1

In [142]:
model = ANN(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters() , lr = learning_rate)

In [143]:
for epoch in range(epochs):
    totalepoch_loss = 0

    for batch_features, batch_labels in train_loader:
        output = model(batch_features)
        loss = criterion(output, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        totalepoch_loss = totalepoch_loss + loss.item()

    avg_loss = totalepoch_loss / len(train_loader)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.6f}")

Epoch: 1, Loss: 1.350902
Epoch: 2, Loss: 0.786678
Epoch: 3, Loss: 0.659856
Epoch: 4, Loss: 0.602403
Epoch: 5, Loss: 0.542793
Epoch: 6, Loss: 0.501532
Epoch: 7, Loss: 0.473812
Epoch: 8, Loss: 0.450119
Epoch: 9, Loss: 0.442032
Epoch: 10, Loss: 0.409061
Epoch: 11, Loss: 0.389693
Epoch: 12, Loss: 0.378582
Epoch: 13, Loss: 0.372092
Epoch: 14, Loss: 0.345504
Epoch: 15, Loss: 0.338395
Epoch: 16, Loss: 0.315893
Epoch: 17, Loss: 0.308681
Epoch: 18, Loss: 0.309420
Epoch: 19, Loss: 0.289865
Epoch: 20, Loss: 0.269798
Epoch: 21, Loss: 0.263540
Epoch: 22, Loss: 0.259489
Epoch: 23, Loss: 0.262790
Epoch: 24, Loss: 0.248780
Epoch: 25, Loss: 0.228659
Epoch: 26, Loss: 0.232720
Epoch: 27, Loss: 0.224315
Epoch: 28, Loss: 0.219278
Epoch: 29, Loss: 0.228014
Epoch: 30, Loss: 0.201469
Epoch: 31, Loss: 0.200214
Epoch: 32, Loss: 0.186682
Epoch: 33, Loss: 0.183886
Epoch: 34, Loss: 0.192472
Epoch: 35, Loss: 0.199524
Epoch: 36, Loss: 0.162845
Epoch: 37, Loss: 0.170136
Epoch: 38, Loss: 0.151237
Epoch: 39, Loss: 0.15

In [144]:
model.eval()

ANN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [145]:
correct = 0
total = 0
with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)


0.8525
